In [1]:
import random
from math import exp
import numpy as np
from scipy import linalg

In [4]:
# ground truth coefficients
ar = 1.0
br = 2.0
cr = 1.0

# initial estimates
ae = 2.0
be = -1.0
ce = 5.0

# number of data points
N = 100

w_sigma = 1.0 # noise sigma
inv_sigma = 1.0 # w_sigma
sig_sq = inv_sigma ** 2

# create data
x_data, y_data = [], []
for i in range(N):
    x = i / 100.0
    x_data.append(x)
    y_data.append(exp(ar * x ** 2 + br * x + cr) + random.gauss(0, w_sigma ** 2))

In [8]:
# Start Gauss-Newton iterations
iterations = 100
cost = 0
lastCost = 0

for iters in range(iterations):
    H = np.zeros((3, 3))  # Hessian = J^T W^-1 J in Gauss-Newton
    b = np.zeros((3, 1))
    cost = 0

    for i in range(N):
        xi = x_data[i]
        yi = y_data[i]

        fun = exp((ae * xi ** 2) + (be * xi) + ce) # function, repeated b/c of chain rule, DRY
        error = yi - fun
        deda = -xi * xi * fun
        dedb = -xi * fun
        dedc = -fun
        J = np.array([deda, dedb, dedc], dtype = np.float32).reshape((3,1))

        H += sig_sq * J * J.T
        b += -sig_sq * error * J

        cost += error ** 2

    # Solve

    dx = linalg.solve(H, b, assume_a='sym')

    if dx[0] is np.NAN:
        print("Result is NaN!")
        break

    if iters > 0 and cost < lastCost:
        print(f"Cost: {cost} last cost: {lastCost}, iter: {iters}, break.")
        break


    ae += dx[0]
    be += dx[1]
    ce += dx[2]

    lastCost = cost

    print(f"total cost: {cost}\nupdate: {dx.transpose()}\n"
            f"estimated params: {ae}, {be}, {ce}")
print(f"estimated abc = {ae}, {be}, {ce}")

total cost: 376838.50239804015
update: [[ 0.06477802  0.22069698 -0.95919334]]
estimated params: [2.10910967], [-0.70204471], [3.05666914]
Cost: 35629.477431077146 last cost: 376838.50239804015, iter: 1, break.
estimated abc = [2.10910967], [-0.70204471], [3.05666914]
